# 1. Data collection

__In this first section, I will take care of the collection of songs from the Spotifycharts playlist offered by Spotify. This phase will be divided into two main steps:__
1. scraping the daily top 200 playlist from the spotifycharts website to get the list of songs and corresponding          artist, date, Song ID, and streams. 
2. connecting to the Spotify API to get the song features through the song IDs found in step 1. 

In [ ]:
#Importing the packages that I will need for this section

#for the scraping step:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from time import sleep
from datetime import date, timedelta

#for the Spotify API step:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#some extra useful libraries
import pandas as pd
import json
import pprint

# 1.1. Web Scraping

__In this step, the expected outcome is a dataframe containing all the songs from the top 200 playlist, including the artist name, the title, the songid, the date and the streams, from 1/12/2018 until the 1/12/2020. The data from 2018-2019 will turn useful later when I will check for seasonality.__

__In order to obtain this, I need to set up the model. In particular, :__

In [41]:
#1. I create three empty lists for data we're collecting
dates=[]
url_list=[]
final = []

In [42]:
#2. I map the site
url = "https://spotifycharts.com/regional/it/daily/"

In [ ]:
# 3. I choose this time frame to track the music trends of the last two years. 
start_date= date(2018,12, 1)
end_date= date(2020, 12, 1)

In [43]:
#4. I create a list of the days corresponding to the time range I am interested of.

delta = end_date-start_date

for i in range(delta.days+1): #for i in the range of delta
    day = start_date+timedelta(days=i) #the day is the start_date + the next day i=0 on the dates list 
    day_string= day.strftime("%Y-%m-%d")# convert the day into this format
    dates.append(day_string) #append the day string to the empty dates list

In [44]:
dates

['2018-12-01',
 '2018-12-02',
 '2018-12-03',
 '2018-12-04',
 '2018-12-05',
 '2018-12-06',
 '2018-12-07',
 '2018-12-08',
 '2018-12-09',
 '2018-12-10',
 '2018-12-11',
 '2018-12-12',
 '2018-12-13',
 '2018-12-14',
 '2018-12-15',
 '2018-12-16',
 '2018-12-17',
 '2018-12-18',
 '2018-12-19',
 '2018-12-20',
 '2018-12-21',
 '2018-12-22',
 '2018-12-23',
 '2018-12-24',
 '2018-12-25',
 '2018-12-26',
 '2018-12-27',
 '2018-12-28',
 '2018-12-29',
 '2018-12-30',
 '2018-12-31',
 '2019-01-01',
 '2019-01-02',
 '2019-01-03',
 '2019-01-04',
 '2019-01-05',
 '2019-01-06',
 '2019-01-07',
 '2019-01-08',
 '2019-01-09',
 '2019-01-10',
 '2019-01-11',
 '2019-01-12',
 '2019-01-13',
 '2019-01-14',
 '2019-01-15',
 '2019-01-16',
 '2019-01-17',
 '2019-01-18',
 '2019-01-19',
 '2019-01-20',
 '2019-01-21',
 '2019-01-22',
 '2019-01-23',
 '2019-01-24',
 '2019-01-25',
 '2019-01-26',
 '2019-01-27',
 '2019-01-28',
 '2019-01-29',
 '2019-01-30',
 '2019-01-31',
 '2019-02-01',
 '2019-02-02',
 '2019-02-03',
 '2019-02-04',
 '2019-02-

In [49]:
#5. I create a list of the urls corresponding to each day of the chart.

for date in dates:
        link_string = url+date #add each date from the "dates" list just created to the url. 
        #This will allow me to build the url of each page corresponding to each day of the spotify chart  
        url_list.append(link_string)

In [50]:
url_list

['https://spotifycharts.com/regional/it/daily/2018-12-01',
 'https://spotifycharts.com/regional/it/daily/2018-12-02',
 'https://spotifycharts.com/regional/it/daily/2018-12-03',
 'https://spotifycharts.com/regional/it/daily/2018-12-04',
 'https://spotifycharts.com/regional/it/daily/2018-12-05',
 'https://spotifycharts.com/regional/it/daily/2018-12-06',
 'https://spotifycharts.com/regional/it/daily/2018-12-07',
 'https://spotifycharts.com/regional/it/daily/2018-12-08',
 'https://spotifycharts.com/regional/it/daily/2018-12-09',
 'https://spotifycharts.com/regional/it/daily/2018-12-10',
 'https://spotifycharts.com/regional/it/daily/2018-12-11',
 'https://spotifycharts.com/regional/it/daily/2018-12-12',
 'https://spotifycharts.com/regional/it/daily/2018-12-13',
 'https://spotifycharts.com/regional/it/daily/2018-12-14',
 'https://spotifycharts.com/regional/it/daily/2018-12-15',
 'https://spotifycharts.com/regional/it/daily/2018-12-16',
 'https://spotifycharts.com/regional/it/daily/2018-12-17

__Second, I proceed with scraping the spotify charts playlist, filtered for Italy, on a daily basis.__

__In order to obtain this, I need to:__

      1. define an html scraping function called song_scrape
      2. loop through the url list to use the song_scrape function in every daily page of the spotify chart.
      3. convert the results into a panda data frame for easier data manipulation
    

In [9]:
def song_scrape(x):
    pg = x
    for tr in songs.find("tbody").findAll("tr"):
        artist= tr.find("td", {"class": "chart-table-track"}).find("span").text
        artist= artist.replace("by ","").strip()
  
        title= tr.find("td", {"class": "chart-table-track"}).find("strong").text
 
        songid= tr.find("td", {"class": "chart-table-image"}).find("a").get("href")
        songid= songid.split("track/")[1]
    
        url_date= x.split("daily/")[1]
        
        streams_text = tr.find('td', {'class': 'chart-table-streams'}).text
        
        final.append([title, artist, songid, url_date, streams_text])

In [11]:
#loop through urls to create array of all of our song info

for u in url_list: #url_list = list of links (spotify daily charts)
    read_pg= requests.get(u) #I send the request to get the url list and I ask the computer to "sleep" in between requests.
    sleep(2)
    soup= BeautifulSoup(read_pg.text, "html.parser") #I use beautiful soup to scrape information in text format.
    songs= soup.find("table", {"class":"chart-table"})#from the html table on the spotify chart website containing all the relevant information.
    song_scrape(u) #I then apply the song_scrape function to scrape the information mentioned above. 

In [12]:
final_df = pd.DataFrame(final, columns= ["Title", "Artist", "Song ID", "Chart Date","Streams"])


__..and here is the first dataframe with all the songs contained in the top 200 daily spotify chart from 1.December 2018 until 1.December 2020.__



In [13]:
final_df

,Title,Artist,Song ID,Chart Date,Streams
0,Happy Birthday,Sfera Ebbasta,7Cw97917dvg5xm6XMAA4Y2,2018-12-01,"775,056"
1,IL CIELO NELLA STANZA (feat. NSTASIA),Salmo,2fkVZYFv9hOejIsLzZy8ad,2018-12-01,"431,785"
2,Torna a casa,Måneskin,3590AAEoqH50z4UmhMIY85,2018-12-01,"349,868"
3,La fine del mondo,"Anastasio, Don Joe",3FhWEh7VTAOYK4edZqNWNB,2018-12-01,"324,392"
4,CABRIOLET (feat. Sfera Ebbasta),Salmo,7wC76LWyNxaP3hjGzbWOcN,2018-12-01,"293,803"
...,...,...,...,...,...
170795,m12ano - feat. Mara Sattei,tha Supreme,0LfMuWu6FKNSDHEoxHu9iK,2020-12-01,"30,677"
170796,Sweater Weather,The Neighbourhood,2QjOHCTQ1Jl3zawyYOpxh6,2020-12-01,"30,526"
170797,BLOODY BARS - DRILLUMINAZIONE (feat. Charlie K...,"BLOODY VINYL, Slait, Young Miles, tha Supreme,...",2RBBYwsf1SKX64nwfbfDlk,2020-12-01,"30,422"
170798,È sempre bello,Coez,6IdiuMw1FSAvU3e6bgWQVX,2020-12-01,"30,132"


__Before moving to the second step of this section, I want to 1) save 2) observe the data just obtained.__

__After a few lines of code I can already check that the data are clean, indeed there are no null values and NAs.
  Moreover, I can notice something interesting: the number of titles are less than the song ID. This evidence is curious as I would expect these two variables to have the same amount of unique values. 
An assumption could be that the same song was made by different artists, for example as remixes, and therefore gets a different ID.__


In [35]:
final_df.to_csv('tracks_with_streams.csv', index = True) 

In [15]:
final_df['Artist'].value_counts()

Ultimo                                             6426
Salmo                                              4736
tha Supreme                                        3428
Capo Plaza                                         3249
Lazza                                              2803
                                                   ... 
Ski & Wok, nemo, DOD                                  1
Jax Jones, Martin Solveig, Madison Beer, Europa       1
Big Fish, Nitro, Nerone                               1
Kanye West, Sunday Service Choir                      1
Oni One                                               1
Name: Artist, Length: 930, dtype: int64

In [36]:
final_df.isna().sum()

Title         0
Artist        0
Song ID       0
Chart Date    0
Streams       0
dtype: int64

In [37]:
final_df.describe()

,Title,Artist,Song ID,Chart Date,Streams
count,2971,2971,2971,2971,2971
unique,2534,930,2971,493,2931
top,Boogieman (feat. Salmo),Lazza,0pqnGHJpmpxLKifKRmU6WP,2018-12-01,"44,083"
freq,7,52,1,200,2


In [16]:
final_df.nunique()   

Title          2534
Artist          930
Song ID        2971
Chart Date      732
Streams       80388
dtype: int64

__Finally, I will drop the Song ID duplicates. This will turn useful for the next section, when I will have to use the Spotify API to obtain the features for each song.
Deleting the duplicates will allow me to send less requests and be more efficient.__

In [19]:
final_drop = final_df #First, I create a new checkpoint. 

In [21]:
final_drop.drop_duplicates(subset=['Song ID'], inplace=True)

In [22]:
final_drop.nunique()

Title         2534
Artist         930
Song ID       2971
Chart Date     493
Streams       2931
dtype: int64

In [23]:
final_drop #Good result. From 170800 rows, I have now only 2971.

,Title,Artist,Song ID,Chart Date,Streams
0,Happy Birthday,Sfera Ebbasta,7Cw97917dvg5xm6XMAA4Y2,2018-12-01,"775,056"
1,IL CIELO NELLA STANZA (feat. NSTASIA),Salmo,2fkVZYFv9hOejIsLzZy8ad,2018-12-01,"431,785"
2,Torna a casa,Måneskin,3590AAEoqH50z4UmhMIY85,2018-12-01,"349,868"
3,La fine del mondo,"Anastasio, Don Joe",3FhWEh7VTAOYK4edZqNWNB,2018-12-01,"324,392"
4,CABRIOLET (feat. Sfera Ebbasta),Salmo,7wC76LWyNxaP3hjGzbWOcN,2018-12-01,"293,803"
...,...,...,...,...,...
170336,Happy Xmas (War Is Over) - Remastered 2010,"John Lennon, Yoko Ono",27RYrbL6S02LNVhDWVl38b,2020-11-29,"37,373"
170382,Happy Xmas (War Is Over),Céline Dion,2ohR9Qj4TgZafZCptO9cly,2020-11-29,"32,152"
170398,Sleigh Ride,The Ronettes,46UMwpkxyIwzqhsaONm4wf,2020-11-29,"30,375"
170582,HOLIDAY,Lil Nas X,6zFMeegAMYQo0mt8rXtrli,2020-11-30,"31,712"


In [24]:
tracks_df = final_drop # I create a second checkpoint.

In [25]:
tracks_df

,Title,Artist,Song ID,Chart Date,Streams
0,Happy Birthday,Sfera Ebbasta,7Cw97917dvg5xm6XMAA4Y2,2018-12-01,"775,056"
1,IL CIELO NELLA STANZA (feat. NSTASIA),Salmo,2fkVZYFv9hOejIsLzZy8ad,2018-12-01,"431,785"
2,Torna a casa,Måneskin,3590AAEoqH50z4UmhMIY85,2018-12-01,"349,868"
3,La fine del mondo,"Anastasio, Don Joe",3FhWEh7VTAOYK4edZqNWNB,2018-12-01,"324,392"
4,CABRIOLET (feat. Sfera Ebbasta),Salmo,7wC76LWyNxaP3hjGzbWOcN,2018-12-01,"293,803"
...,...,...,...,...,...
170336,Happy Xmas (War Is Over) - Remastered 2010,"John Lennon, Yoko Ono",27RYrbL6S02LNVhDWVl38b,2020-11-29,"37,373"
170382,Happy Xmas (War Is Over),Céline Dion,2ohR9Qj4TgZafZCptO9cly,2020-11-29,"32,152"
170398,Sleigh Ride,The Ronettes,46UMwpkxyIwzqhsaONm4wf,2020-11-29,"30,375"
170582,HOLIDAY,Lil Nas X,6zFMeegAMYQo0mt8rXtrli,2020-11-30,"31,712"


# 1.2. Spotify API

__In this second section, the expected outcome is a dataframe containing all the song features that I will be able to obtain by sending requests to the Spotify API through each Song ID scraped in the previous section.__

__In order to obtain this, I will have to:__ 

    1. Initialize the Spotipy with my user credentials, 
    2. Set up a for loop to send requests to the Spotify API to download the song features.
       As the API sets a limit of 100 tracks per request, I made sure that the loop proceeded as such.
       

In [29]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id="ad258b4145944bf683bea0a7ceac71c8",
    client_secret="7563fb1e63f44e89bb9f1d55eeefe9c0"))

In [31]:
features_dfs = []
count = 0
for i in range(30): #repeating the loop 30 times in bulks of 100 will get the loop to obtain the features for 2971 songs.
    features = pd.DataFrame(sp.audio_features(tracks_df['Song ID'][count:count+100])) 
    features_dfs.append(features)
    count+=100 #the loop will send a new request (i.e. will start again) every 100 features 
    
track_features = pd.concat(features_dfs)

In [32]:
track_features 

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.792,0.777,8,-4.402,0,0.0454,0.0455,0.000053,0.2930,0.696,137.050,audio_features,7Cw97917dvg5xm6XMAA4Y2,spotify:track:7Cw97917dvg5xm6XMAA4Y2,https://api.spotify.com/v1/tracks/7Cw97917dvg5...,https://api.spotify.com/v1/audio-analysis/7Cw9...,170707,4
1,0.755,0.501,1,-9.551,1,0.1890,0.0867,0.000001,0.1460,0.267,134.998,audio_features,2fkVZYFv9hOejIsLzZy8ad,spotify:track:2fkVZYFv9hOejIsLzZy8ad,https://api.spotify.com/v1/tracks/2fkVZYFv9hOe...,https://api.spotify.com/v1/audio-analysis/2fkV...,186213,4
2,0.425,0.638,1,-3.184,0,0.0759,0.4260,0.000000,0.1770,0.450,81.396,audio_features,3590AAEoqH50z4UmhMIY85,spotify:track:3590AAEoqH50z4UmhMIY85,https://api.spotify.com/v1/tracks/3590AAEoqH50...,https://api.spotify.com/v1/audio-analysis/3590...,230667,4
3,0.683,0.622,1,-5.895,0,0.1750,0.0165,0.000000,0.1200,0.328,159.946,audio_features,3FhWEh7VTAOYK4edZqNWNB,spotify:track:3FhWEh7VTAOYK4edZqNWNB,https://api.spotify.com/v1/tracks/3FhWEh7VTAOY...,https://api.spotify.com/v1/audio-analysis/3FhW...,183000,4
4,0.548,0.531,0,-7.436,1,0.2640,0.1500,0.000010,0.0919,0.309,150.058,audio_features,7wC76LWyNxaP3hjGzbWOcN,spotify:track:7wC76LWyNxaP3hjGzbWOcN,https://api.spotify.com/v1/tracks/7wC76LWyNxaP...,https://api.spotify.com/v1/audio-analysis/7wC7...,185573,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,0.321,0.640,2,-10.023,1,0.0324,0.2800,0.000000,0.7180,0.414,146.172,audio_features,27RYrbL6S02LNVhDWVl38b,spotify:track:27RYrbL6S02LNVhDWVl38b,https://api.spotify.com/v1/tracks/27RYrbL6S02L...,https://api.spotify.com/v1/audio-analysis/27RY...,214333,3
67,0.347,0.600,3,-5.254,1,0.0303,0.4510,0.000000,0.2790,0.158,172.062,audio_features,2ohR9Qj4TgZafZCptO9cly,spotify:track:2ohR9Qj4TgZafZCptO9cly,https://api.spotify.com/v1/tracks/2ohR9Qj4TgZa...,https://api.spotify.com/v1/audio-analysis/2ohR...,254707,4
68,0.529,0.772,2,-7.013,1,0.0287,0.4030,0.000002,0.3160,0.853,91.751,audio_features,46UMwpkxyIwzqhsaONm4wf,spotify:track:46UMwpkxyIwzqhsaONm4wf,https://api.spotify.com/v1/tracks/46UMwpkxyIwz...,https://api.spotify.com/v1/audio-analysis/46UM...,181267,4
69,0.810,0.511,5,-6.924,0,0.1640,0.1200,0.000000,0.0832,0.837,151.947,audio_features,6zFMeegAMYQo0mt8rXtrli,spotify:track:6zFMeegAMYQo0mt8rXtrli,https://api.spotify.com/v1/tracks/6zFMeegAMYQo...,https://api.spotify.com/v1/audio-analysis/6zFM...,154998,4


In [33]:
track_features.to_csv('features_with_.csv', index = True) 

In [34]:
track_features.shape

(2971, 18)